# ディープラーニングによる自然言語処理: 評判分析モデルの実装

このノートブックでは、ディープラーニングを用いた自然言語処理のためのツールである[AllenNLP](https://allennlp.org/)を用いて、**畳み込みニューラルネットワーク（CNN）**を用いた**評判分析**のシステムを作成します。

## 環境のセットアップ

In [1]:
# AllenNLPをインストール
!pip install allennlp==2.9.3 google-cloud-storage==2.1.0 cached-path==1.1.2
# fugashiをUniDicの依存ライブラリを含めてインストール
!pip install fugashi[unidic]
# UniDicの辞書ファイルをダウンロード
!python -m unidic download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.6/719.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 27.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... d

## データセットのセットアップ

In [2]:
# データセットの出力ディレクトリを作成
!mkdir -p data/amazon_reviews
# データセットをダウンロード
!wget -q -O data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv.gz "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz"
# データセットを解凍し、data/amazon_reviewsに展開
!gunzip data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv.gz

In [3]:
import csv
import json
import os
import random
import warnings
from bs4 import BeautifulSoup

# csvライブラリのフィールドの最大サイズを変更
csv.field_size_limit(1000000)
# BeautifulSoupの出力する警告を抑制
warnings.filterwarnings("ignore", category=UserWarning, module="bs4")

# データセットをファイルから読み込む
data = []
with open("data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv") as f:
    reader = csv.reader(f, delimiter="\t")
    # 1行目はヘッダなので無視する
    next(reader)
    for r in reader:
        # レビューのテキストを取得
        review_body = r[13]
        # レビューのテキストからHTMLタグを除去
        review_body = BeautifulSoup(review_body, "html.parser").get_text()
        # 評価の値を取得
        ratings = int(r[7])
        # 評価が2以下の場合に否定的、4以上の場合に肯定的と扱う
        if ratings <= 2:
            data.append(dict(text=review_body, label="negative"))
        elif ratings >= 4:
            data.append(dict(text=review_body, label="positive"))

# データセットから50,000件をランダムに抽出する
random.seed(1)
random.shuffle(data)
data = data[:50000]

# データセットの80%を訓練データ、10%を検証データ、10%をテストデータとして用いる
split_data = {}
eval_size = int(len(data) * 0.1)
split_data["test"] = data[:eval_size]
split_data["validation"] = data[eval_size:eval_size * 2]
split_data["train"] = data[eval_size * 2:]

# 行区切りJSON形式でデータセットを書き込む
for fold in ("train", "validation", "test"):
    out_file = os.path.join("data/amazon_reviews", "amazon_reviews_{}.jsonl".format(fold))
    with open(out_file, mode="w") as f:
        for item in split_data[fold]:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")

<ipython-input-3-afdd09eabd0a>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_body = BeautifulSoup(review_body, "html.parser").get_text()
<ipython-input-3-afdd09eabd0a>:23: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_body = BeautifulSoup(review_body, "html.parser").get_text()


In [4]:
!ls data/amazon_reviews/*.jsonl

data/amazon_reviews/amazon_reviews_test.jsonl
data/amazon_reviews/amazon_reviews_train.jsonl
data/amazon_reviews/amazon_reviews_validation.jsonl


## モデルの実装

In [5]:
model_config = """{
    "random_seed": 1,
    "pytorch_seed": 1,
    "train_data_path": "data/amazon_reviews/amazon_reviews_train.jsonl",
    "validation_data_path": "data/amazon_reviews/amazon_reviews_validation.jsonl",
    "dataset_reader": {
        "type": "text_classification_json",
        "tokenizer": {
            "type": "mecab"
        },
        "token_indexers": {
            "tokens": {
                "type": "single_id"
            }
        }
    },
    "data_loader": {
        "batch_size": 32,
        "shuffle": true
    },
    "validation_data_loader": {
        "batch_size": 32,
        "shuffle": false
    },
    "vocabulary": {},
    "datasets_for_vocab_creation": ["train"],
    "model": {
        "type": "basic_classifier",
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 100
                }
            }
        },
        "seq2vec_encoder": {
           "type": "cnn",
           "embedding_dim": 100,
           "ngram_filter_sizes": [2],
           "num_filters": 64,
           "conv_layer_activation": "relu"
        }
    },
    "trainer": {
        "cuda_device": 0,
        "optimizer": {
            "type": "adam"
        },
        "num_epochs": 10,
        "patience": 3,
        "callbacks": [
            {
                "type": "tensorboard"
            }
        ]
    }
}"""
with open("amazon_reviews.jsonnet", "w") as f:
  f.write(model_config)

## モデルの訓練

In [6]:
mecab_tokenizer_code = '''from allennlp.data.tokenizers.token_class import Token
from allennlp.data.tokenizers.tokenizer import Tokenizer
from fugashi import Tagger

@Tokenizer.register("mecab")
class MecabTokenizer(Tokenizer):
    def __init__(self):
        # Taggerインスタンスを作成
        self._tagger = Tagger()

    def tokenize(self, text):
        """入力テキストをMeCabを用いて解析する"""
        tokens = []
        # 入力テキストを単語に分割
        for word in self._tagger(text):
            # 単語のテキスト（word.surface）と品詞（word.feature.pos1)からTokenインスタンスを作成
            token = Token(text=word.surface, pos_=word.feature.pos1)
            tokens.append(token)

        return tokens
'''
with open("mecab_tokenizer.py", "w") as f:
  f.write(mecab_tokenizer_code)

In [7]:
!allennlp train --serialization-dir exp_amazon_reviews --include-package mecab_tokenizer amazon_reviews.jsonnet

2023-05-06 13:29:28.638335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 13:29:31.069630: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-06 13:29:37,947 - INFO - allennlp.common.params - evaluation = None
2023-05-06 13:29:37,947 - INFO - allennlp.common.params - include_in_archive = None
2023-05-06 13:29:37,947 - INFO - allennlp.common.params - random_seed = 1
2023-05-06 13:29:37,948 - INFO - allennlp.common.params - numpy_seed = 1337
2023-05-06 13:29:37,948 - INFO - allennlp.common.params - pytorch_seed = 1
2023-05-06 13:29:37,949 - INFO - allennlp.common.checks - Pytorch version: 1.11.0+cu102
2023-05-06 13:29:37,949 - INFO - allennlp.common.params - type = default
2023-05-06 13:29:3

## 性能の評価

In [8]:
!allennlp evaluate --include-package mecab_tokenizer exp_amazon_reviews/model.tar.gz data/amazon_reviews/amazon_reviews_test.jsonl

2023-05-06 13:32:21.488361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 13:32:22.448457: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-06 13:32:26,870 - INFO - allennlp.models.archival - loading archive file exp_amazon_reviews/model.tar.gz
2023-05-06 13:32:26,870 - INFO - allennlp.models.archival - extracting archive file exp_amazon_reviews/model.tar.gz to temp dir /tmp/tmp9fgyq33p
2023-05-06 13:32:27,336 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmp9fgyq33p/vocabulary.
2023-05-06 13:32:27,655 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /tmp/tmp9fgyq33p
2023-05-06 13:32:27,666 - INFO - allennlp.common.checks - Pytorch v

## 学習したモデルを使う

In [9]:
input_json = """{"sentence":"この本は、役に立つし、面白い。"}
{"sentence":"この本は、役に立たないし、面白くない。"}
{"sentence":"この本は、役に立たないけど、面白い。"}
"""
with open("sentiment_analysis_input.json", "w") as f:
  f.write(input_json)

In [10]:
!allennlp predict --include-package mecab_tokenizer exp_amazon_reviews/model.tar.gz sentiment_analysis_input.json

2023-05-06 13:32:47.401271: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 13:32:48.909860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-06 13:32:53,083 - INFO - allennlp.models.archival - loading archive file exp_amazon_reviews/model.tar.gz
2023-05-06 13:32:53,083 - INFO - allennlp.models.archival - extracting archive file exp_amazon_reviews/model.tar.gz to temp dir /tmp/tmprgqp0m3q
2023-05-06 13:32:53,437 - INFO - allennlp.common.params - dataset_reader.type = text_classification_json
2023-05-06 13:32:53,438 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2023-05-06 13:32:53,438 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = Fals

### ハイパーパラメータ探索

In [11]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.3 MB/s eta 0:00:00


In [12]:
optuna_config = """local embedding_dim = std.parseInt(std.extVar("embedding_dim"));
local num_filters = std.parseInt(std.extVar("num_filters"));
local max_filter_size = std.parseInt(std.extVar("max_filter_size"));
local ngram_filter_sizes = std.range(2, max_filter_size);
{
    "random_seed": 1,
    "pytorch_seed": 1,
    "train_data_path": "data/amazon_reviews/amazon_reviews_train.jsonl",
    "validation_data_path": "data/amazon_reviews/amazon_reviews_validation.jsonl",
    "dataset_reader": {
        "type": "text_classification_json",
        "tokenizer": {
            "type": "mecab"
        },
        "token_indexers": {
            "tokens": {
                "type": "single_id"
            }
        }
    },
    "data_loader": {
        "batch_size": 32,
        "shuffle": true
    },
    "validation_data_loader": {
        "batch_size": 32,
        "shuffle": false
    },
    "vocabulary": {},
    "datasets_for_vocab_creation": ["train"],
    "model": {
        "type": "basic_classifier",
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": embedding_dim
                }
            }
        },
        "seq2vec_encoder": {
           "type": "cnn",
           "embedding_dim": embedding_dim,
           "ngram_filter_sizes": ngram_filter_sizes,
           "num_filters": num_filters,
           "conv_layer_activation": "relu"
        }
    },
    "trainer": {
        "cuda_device": 0,
        "optimizer": {
            "type": "adam"
        },
        "num_epochs": 10,
        "patience": 3
    }
}
"""
with open("amazon_reviews_optuna.jsonnet", "w") as f:
  f.write(optuna_config)

In [13]:
import optuna
from optuna.samplers import TPESampler
from optuna.integration.allennlp import AllenNLPExecutor, dump_best_config

def objective(trial):
    """Optunaの目的関数"""
    # 探索するハイパーパラメータと値の範囲を定義
    trial.suggest_int("embedding_dim", 50, 200)
    trial.suggest_int("max_filter_size", 2, 5)
    trial.suggest_int("num_filters", 32, 256)

    # AllenNLPExecutorを作成し訓練と評価を行う
    serialization_dir = "exp_amazon_reviews_optuna/trials/" + str(trial.number)
    executor = AllenNLPExecutor(
        trial=trial,
        metrics="best_validation_accuracy",
        serialization_dir=serialization_dir,
        config_file="amazon_reviews_optuna.jsonnet",
        include_package="mecab_tokenizer"
    )
    return executor.run()

In [14]:
!mkdir -p exp_amazon_reviews_optuna

In [15]:
# ハイパーパラメータ探索を開始
study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=1))
study.optimize(objective, n_trials=100)

# 最も性能の良かったハイパーパラメータを用いて設定ファイルを作成
dump_best_config("amazon_reviews_optuna.jsonnet", "amazon_reviews_best.json", study)

[I 2023-05-06 13:33:11,658] A new study created in memory with name: no-name-763c30fc-732b-448c-ac7a-30cf3735bae2


2023-05-06 13:33:11,724 - INFO - allennlp.common.params - random_seed = 1
2023-05-06 13:33:11,729 - INFO - allennlp.common.params - numpy_seed = 1337
2023-05-06 13:33:11,730 - INFO - allennlp.common.params - pytorch_seed = 1
2023-05-06 13:33:11,735 - INFO - allennlp.common.checks - Pytorch version: 1.11.0+cu102
2023-05-06 13:33:11,737 - INFO - allennlp.common.params - type = default
2023-05-06 13:33:11,740 - INFO - allennlp.common.params - dataset_reader.type = text_classification_json
2023-05-06 13:33:11,743 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2023-05-06 13:33:11,745 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2023-05-06 13:33:11,747 - INFO - allennlp.common.params - dataset_reader.manual_multiprocess_sharding = False
2023-05-06 13:33:11,750 - INFO - allennlp.common.params - dataset_reader.token_indexers.tokens.type = single_id
2023-05-06 13:33:11,751 - INFO - allennlp.common.params - dataset_reader.token_index

<ipython-input-13-ee19242cc6db>:14: ExperimentalWarning: AllenNLPExecutor is experimental (supported from v1.4.0). The interface can change in the future.
  executor = AllenNLPExecutor(
loading instances: 40000it [01:22, 482.05it/s]

2023-05-06 13:34:34,778 - INFO - allennlp.common.params - validation_data_loader.type = multiprocess
2023-05-06 13:34:34,782 - INFO - allennlp.common.params - validation_data_loader.batch_size = 32


2023-05-06 13:34:34,784 - INFO - allennlp.common.params - validation_data_loader.drop_last = False
2023-05-06 13:34:34,786 - INFO - allennlp.common.params - validation_data_loader.shuffle = False
2023-05-06 13:34:34,792 - INFO - allennlp.common.params - validation_data_loader.batch_sampler = None
2023-05-06 13:34:34,795 - INFO - allennlp.common.params - validation_data_loader.batches_per_epoch = None
2023-05-06 13:34:34,796 - INFO - allennlp.common.params - validation_data_loader.num_workers = 0
2023-05-06 13:34:34,799 - INFO - allennlp.common.params - validation_data_loader.max_instances_in_memory = None
2023-05-06 13:34:34,800 - INFO - allennlp.common.params - validation_data_loader.start_method = fork
2023-05-06 13:34:34,803 - INFO - allennlp.common.params - validation_data_loader.cuda_device = None
2023-05-06 13:34:34,806 - INFO - allennlp.common.params - validation_data_loader.quiet = False
2023-05-06 13:34:34,807 - INFO - allennlp.common.params - validation_data_loader.collate_fn

...

2023-05-06 20:27:35,875 - INFO - allennlp.training.gradient_descent_trainer - Validating



accuracy: 0.9254, batch_loss: 0.0027, loss: 0.3618 ||: 100%|##########| 157/157 [00:01<00:00, 130.74it/s]

2023-05-06 20:27:37,084 - INFO - allennlp.training.callbacks.console_logger -                        Training |  Validation
2023-05-06 20:27:37,087 - INFO - allennlp.training.callbacks.console_logger - accuracy           |     1.000  |     0.925
2023-05-06 20:27:37,089 - INFO - allennlp.training.callbacks.console_logger - gpu_0_memory_MB    |  1172.501  |       N/A
2023-05-06 20:27:37,090 - INFO - allennlp.training.callbacks.console_logger - loss               |     0.002  |     0.362
2023-05-06 20:27:37,092 - INFO - allennlp.training.callbacks.console_logger - worker_0_memory_MB |  68356.492  |       N/A


2023-05-06 20:27:38,321 - INFO - allennlp.training.gradient_descent_trainer - Epoch duration: 0:00:32.235845
2023-05-06 20:27:38,323 - INFO - allennlp.training.gradient_descent_trainer - Ran out of patience. Stopping training.
2023-05-06 20:27:38,453 - INFO - allennlp.common.util - Metrics: {
  "best_epoch": 1,
  "peak_worker_0_memory_MB": 68356.4921875,
  "peak_gpu_0_memory_MB": 1172.50146484375,
  "training_duration": "0:02:40.176911",
  "epoch": 4,
  "training_accuracy": 0.9997,
  "training_loss": 0.00201418146918877,
  "training_worker_0_memory_MB": 68356.4921875,
  "training_gpu_0_memory_MB": 1172.50146484375,
  "validation_accuracy": 0.9254,
  "validation_loss": 0.36179180754271517,
  "best_validation_accuracy": 0.9328,
  "best_validation_loss": 0.18694678494101688
}
2023-05-06 20:27:38,455 - INFO - allennlp.models.archival - archiving weights and vocabulary to exp_amazon_reviews_optuna/trials/99/model.tar.gz


[I 2023-05-06 20:27:43,423] Trial 99 finished with value: 0.9328 and parameters: {'embedding_dim': 184, 'max_filter_size': 5, 'num_filters': 52}. Best is trial 87 with value: 0.936.


In [16]:
!cat amazon_reviews_best.json

{
    "data_loader": {
        "batch_size": 32,
        "shuffle": true
    },
    "dataset_reader": {
        "token_indexers": {
            "tokens": {
                "type": "single_id"
            }
        },
        "tokenizer": {
            "type": "mecab"
        },
        "type": "text_classification_json"
    },
    "datasets_for_vocab_creation": [
        "train"
    ],
    "model": {
        "seq2vec_encoder": {
            "conv_layer_activation": "relu",
            "embedding_dim": 149,
            "ngram_filter_sizes": [
                2,
                3,
                4,
                5
            ],
            "num_filters": 55,
            "type": "cnn"
        },
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "embedding_dim": 149,
                    "type": "embedding"
                }
            }
        },
        "type": "basic_classifier"
    },
    "pytorch_seed": 1,
    "rando